# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [122]:
## Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from pandasql import sqldf
from pprint import pprint
import scipy.stats as st

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [123]:
## Establish Dataframe from CSV output of Part 1 WeatherPy
output_data_file = "output_data/weather_df.csv"
weatherdf = pd.read_csv(output_data_file)
weatherdf

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,butaritari,100,KI,1640582852,75,3.0707,172.7902,28.07,5.05
1,1,ushuaia,75,AR,1640582750,57,-54.8000,-68.3000,8.81,6.17
2,2,keflavik,75,IS,1640582854,80,64.0049,-22.5624,-0.63,1.54
3,3,hermanus,51,ZA,1640582855,88,-34.4187,19.2345,17.72,3.69
4,4,saint-philippe,82,RE,1640582821,87,-21.3585,55.7679,27.75,5.84
...,...,...,...,...,...,...,...,...,...,...
551,551,bonavista,79,CA,1640583129,97,48.6499,-53.1147,1.73,10.70
552,552,riyadh,100,SA,1640583284,52,24.6877,46.7219,10.08,2.56
553,553,cartagena del chaira,20,CO,1640583286,89,10.3997,-75.5144,25.79,1.54
554,554,talcahuano,0,CL,1640583015,82,-36.7167,-73.1167,15.82,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [124]:
## Configure gmaps
gmaps.configure(api_key=g_key)

In [125]:
## Establish lat_lon variable to contain latitude and longitude data
lat_lon = weatherdf[['Lat', 'Lng']]

### Establish humidity variable to contain Humidity data
humidity = weatherdf["Humidity"]

## Set up sizing for heat map
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '3px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
}

## Use gmaps.figure to display fig correctly
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

## Add heat layer to map 
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity, max_intensity=100)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='500px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [126]:
## Get query which produces data frame with above conditions (NB: I set my API to retrieve Celsius therefore I have amended the query to reflect that condition) )
pysqldf = lambda q: sqldf(q, globals())
weather_query = "SELECT weatherdf.[City],weatherdf.[Country],weatherdf.[Lat],weatherdf.[Lng] FROM weatherdf where (weatherdf.[Max Temp] >= 20 AND weatherdf.[Max Temp] <= 27)  and weatherdf.[Wind Speed] < 10 and weatherdf.[Cloudiness] = 0 order by weatherdf.[Max Temp] desc"

## Print Norther Hemisphere Data Frame
best_weather = pysqldf(weather_query)
best_weather


,City,Country,Lat,Lng
0,santiago del estero,CL,-33.4569,-70.6483
1,salta,AR,-24.7859,-65.4117
2,praya,CV,14.9215,-23.5087
3,praia,CV,14.9215,-23.5087
4,river falls,LK,6.7613,80.7910
5,rawson,AR,-43.3002,-65.1023
6,champoton,MX,19.3500,-90.7167


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [127]:
## Create hotel_df Dataframe and establibish empty column to be filled with API pull of Hotel Names
hotel_df = pd.DataFrame(best_weather, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""

## Establibish lat_lng function to contain location coordinates of city from hotel_df
lat_lng1 = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"

## params dict to to iteratively update hotel_n
params = {
    "location": lat_lng1,
    "radius": 5000,
    "keyword": "hotel",
    "types": "hotel",
    "key": g_key
}

## Establish Base url variable
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Send request using params dict
response = requests.get(base_url, params=params)

## Print response json
pprint(response.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'Aap_uEArb_hS6LYUs2b48qSmgs9qW9MYqfZW4Vyai58i53eMKKnqSUj-GcPNK2TQlm5rDFgUVodGa66sfXEhS28t4XVNcO7yCzab4l1UL6ymINn7Kz0i57ZxDvQ5KV4UU6n1mUn51a43HdpdxfHEXewMKcvhJReDODreV4HQjY64ZpnGptLu2oQgHsyPDq6qqW2tLOT7LlVoW2pSLplE9Dd69XxlsmESy-RJgrx0Tukus1-3mqaDO5LMwr2d-3GYraou5a4yV8ngnhXT-HtPmw44yJqGb707Yq3HaIemaQ2E4Uc2mEWtFPLH61hKe891vI0b6yqdaTiCbsvVOR9X6n5D5PoTWn4FxwYjzun0jNoCHR6VVuyPpfozpYheIGg1DoaWG_9LtGjU1j3ToXLtE19mkjzbtelUYMaOA-Xf4coEClbEU7bbmCLLLrlOATnEvS4',
 'results': [...],
 'status': 'OK'}


In [129]:
#creates a list to capture hotel_n to insert into dataframe
hotel_n = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for city, i in hotel_df.iterrows():
    # get lat, lng from df
    lat = i["Lat"]
    lng = i["Lng"]

    ## Iteratevely get the location coordinates 'lat', 'lng'
    params["location"] = f"{lat},{lng}"

    ## Send Request and get base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    
    ## Convert to JSON and handle Index Error exception  
    name_address = name_address.json()
    try:
        hotel_n.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_n.append(np.nan)
        
hotel_n


['Sheraton Santiago Hotel and Convention Center',
 'Sheraton Salta Hotel',
 'Hotel Santiago',
 'Hotel Oasis Atlantico Praiamar',
 'the glenrock Luxury Nature Resort, Ulugalathenna, Ihala Galagama, Belihuloya, Sri Lanka, 70144',
 'Hosteria Sampedro',
 'Hotel Perla del Mar Champotón']

In [128]:
# NOTE: Do not change any of the code in this cell
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**i) for city, i in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]






In [121]:
## Create symbol layer for hotels to overlay map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

## Set up sizing for heat map
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '3px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
}

## Use gmaps.figure to display fig2 correctly
fig2 = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

## Add marker & hotel_layer on top of heat map
fig2.add_layer(markers)
fig2.add_layer(hotel_layer)

## Display fig2
fig2


Figure(layout=FigureLayout(border='3px solid black', height='500px', margin='0 auto 0 auto', padding='3px', wi…